In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
pd.set_option('display.max_columns', None)  # 모든 열 보이기
pd.set_option('display.max_rows', None)     # 모든 행 보이기

In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed()

In [3]:
class CFG:
    max_len = 512
    batch_size = 16
    learning_rate = 2e-5
    epochs = 7
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [4]:
# 데이터 로드
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [5]:
train_df.dropna(inplace=True)
train_df.drop_duplicates(subset=['제목', '키워드'], keep='first', inplace=True)

In [6]:
def normalize_title(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def normalize_keywords(text):
    text = re.sub(r'[^가-힣\s,]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def clean_keywords(keywords):
    keywords = keywords.split(',')
    keywords = [keyword.strip() for keyword in keywords if keyword.strip()]
    return ' '.join(keywords)

train_df['title'] = train_df['제목'].apply(normalize_title)
train_df['keywords'] = train_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
train_df['text'] = train_df['title'] + ' [SEP] ' + train_df['keywords']

test_df['title'] = test_df['제목'].apply(normalize_title)
test_df['keywords'] = test_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
test_df['text'] = test_df['title'] + ' [SEP] ' + test_df['keywords']

In [7]:
def get_length(text):
    return len(text.split())

# 제목, 키워드, 전체 텍스트의 길이 계산
train_df['title_length'] = train_df['title'].apply(get_length)
train_df['keywords_length'] = train_df['keywords'].apply(get_length)
train_df['total_length'] = train_df['text'].apply(get_length)

# 길이 통계 출력
print("제목 길이 통계:")
print(train_df['title_length'].describe())
print("\n키워드 길이 통계:")
print(train_df['keywords_length'].describe())
print("\n전체 텍스트 길이 통계:")
print(train_df['total_length'].describe())

제목 길이 통계:
count    54315.000000
mean         7.444444
std          2.078915
min          1.000000
25%          6.000000
50%          7.000000
75%          9.000000
max         27.000000
Name: title_length, dtype: float64

키워드 길이 통계:
count    54315.000000
mean       165.699199
std        113.597436
min          4.000000
25%         99.000000
50%        142.000000
75%        203.000000
max       2628.000000
Name: keywords_length, dtype: float64

전체 텍스트 길이 통계:
count    54315.000000
mean       174.143644
std        114.068817
min          6.000000
25%        107.000000
50%        150.000000
75%        212.000000
max       2637.000000
Name: total_length, dtype: float64


In [17]:
# 고유한 분류 라벨 추출
unique_labels = train_df['분류'].unique()
label_counts = train_df['분류'].value_counts()

# 정렬 (선택사항)
unique_labels.sort()

# 정렬 (선택사항)
label_counts_sorted = label_counts.sort_values(ascending=False)

# 라벨별 개수 출력
print("라벨별 개수:")
for label, count in label_counts_sorted.items():
    print(f"{label}: {count}")
    
# 고유한 라벨 출력
print("고유한 분류 라벨:")
for i, label in enumerate(unique_labels, 1):
    print(f"{i}. {label}")

# 총 고유 라벨 수 출력
print(f"\n총 고유 라벨 수: {len(unique_labels)}")

라벨별 개수:
지역: 21480
경제:부동산: 2758
사회:사건_사고: 2036
경제:반도체: 1847
사회:사회일반: 1166
사회:교육_시험: 794
정치:국회_정당: 772
사회:의료_건강: 749
경제:취업_창업: 674
스포츠:올림픽_아시안게임: 666
경제:산업_기업: 566
문화:전시_공연: 534
경제:자동차: 502
경제:경제일반: 497
사회:장애인: 497
스포츠:골프: 491
정치:선거: 483
경제:유통: 467
사회:여성: 427
IT_과학:모바일: 426
사회:노동_복지: 353
사회:환경: 317
경제:서비스_쇼핑: 310
경제:무역: 298
정치:행정_자치: 278
문화:방송_연예: 267
국제: 262
경제:금융_재테크: 261
스포츠:축구: 255
정치:청와대: 223
문화:출판: 196
IT_과학:IT_과학일반: 194
IT_과학:인터넷_SNS: 190
문화:미술_건축: 183
정치:정치일반: 177
IT_과학:과학: 172
문화:문화일반: 168
문화:학술_문화재: 160
문화:요리_여행: 147
경제:자원: 142
문화:종교: 138
IT_과학:콘텐츠: 128
사회:미디어: 102
사회:날씨: 95
스포츠:농구_배구: 90
문화:음악: 87
문화:생활: 80
IT_과학:보안: 74
스포츠:월드컵: 71
경제:증권_증시: 59
정치:북한: 54
정치:외교: 25
스포츠:스포츠일반: 23
문화:영화: 21
스포츠:야구: 13
경제:외환: 7
고유한 분류 라벨:
1. IT_과학:IT_과학일반
2. IT_과학:과학
3. IT_과학:모바일
4. IT_과학:보안
5. IT_과학:인터넷_SNS
6. IT_과학:콘텐츠
7. 경제:경제일반
8. 경제:금융_재테크
9. 경제:무역
10. 경제:반도체
11. 경제:부동산
12. 경제:산업_기업
13. 경제:서비스_쇼핑
14. 경제:외환
15. 경제:유통
16. 경제:자동차
17. 경제:자원
18. 경제:증권_증시
19. 경제:취업_창업
20. 국제
21. 문화:문화일반
22. 문화:미술_건

In [9]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 학습 및 검증 데이터 분할
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

In [10]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [11]:
# 토크나이저 및 모델 로드
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator', num_labels=len(label_encoder)).to(CFG.device)

# 데이터셋 및 데이터로더 생성
train_dataset = TextDataset(train_df.text.tolist(), train_df.label.tolist(), tokenizer, CFG.max_len)
val_dataset = TextDataset(val_df.text.tolist(), val_df.label.tolist(), tokenizer, CFG.max_len)
test_dataset = TextDataset(test_df.text.tolist(), None, tokenizer, CFG.max_len)

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

In [12]:
# 학습 및 검증
for epoch in range(CFG.epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epochs}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(CFG.device)
        attention_mask = batch['attention_mask'].to(CFG.device)
        labels = batch['labels'].to(CFG.device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{CFG.epochs}, Train Loss: {train_loss / len(train_loader):.4f}")
    
    # 검증
    model.eval()
    val_predictions = []
    val_true_labels = []
    val_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(CFG.device)
            attention_mask = batch['attention_mask'].to(CFG.device)
            labels = batch['labels'].to(CFG.device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())
    
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

Epoch 1/7: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2716/2716 [16:51<00:00,  2.69it/s]


Epoch 1/7, Train Loss: 1.5357


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 679/679 [01:42<00:00,  6.61it/s]


Validation Loss: 1.0378, Validation Accuracy: 0.7313, Validation F1 Score: 0.2643


Epoch 2/7: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2716/2716 [16:49<00:00,  2.69it/s]


...

Validation Loss: 0.7235, Validation Accuracy: 0.8194, Validation F1 Score: 0.5931


Epoch 7/7: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2716/2716 [16:50<00:00,  2.69it/s]


Epoch 7/7, Train Loss: 0.2592


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 679/679 [01:42<00:00,  6.61it/s]

Validation Loss: 0.7651, Validation Accuracy: 0.8202, Validation F1 Score: 0.6185


In [13]:
# 테스트 예측
model.eval()
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(CFG.device)
        attention_mask = batch['attention_mask'].to(CFG.device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1463/1463 [03:42<00:00,  6.59it/s]


In [16]:
# 예측 결과 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

# 제출 파일 생성
sample_submission = pd.read_csv("data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions
sample_submission.to_csv("submission_koelectra.csv", encoding='UTF-8-sig', index=False)

print("Prediction completed and submission file created.")

Prediction completed and submission file created.
